In [ ]:
def int_rectangle(f, a, b, nb_rect=100):
    somme_sup, somme_inf = 0,0
    largeur_rect = (b-a)/nb_rect
    for i in range(nb_rect):
        somme_inf += f(a + i*largeur_rect) * largeur_rect
        somme_sup += f(a + (i+1)*largeur_rect) * largeur_rect
    return (somme_inf, somme_sup)
    

In [ ]:
def int_trapeze(f, a, b, nb_rect=100):
    somme = 0
    largeur = (b-a)/nb_rect
    for i in range(nb_rect):
        somme += (f(a + i*largeur) + f(a + (i+1)*largeur))/2 * largeur
    return somme

In [ ]:
def int_milieux(f, a, b, nb_rect=100):
    somme = 0
    largeur = (b-a)/nb_rect
    for i in range(nb_rect):
        somme += f(a + (i+1/2) * largeur) * largeur
    return somme

In [1]:
from math import sqrt
import random

def monte_carlo(f, borne_abs, borne_ord, tirs=1000):
    touches = 0
    x0, x1 = borne_abs
    y0, y1 = borne_ord
    for _ in range(tirs):
        x = random.uniform(x0, x1)
        y = random.uniform(y0, y1)
        if abs(y - f(x)) <= 0:
            touches += 1
    return (touches / tirs) * (x1 - x0) * (y1 - y0)

def approx_pi(tirs=1000):
    touches = 0
    for _ in range(tirs):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if sqrt(x**2+y**2) <= 1:
            touches += 1
    return touches / tirs * 4